<a href="https://colab.research.google.com/github/apurba420/Cassava_Leaf_Classification-traing_on_TPU/blob/main/Tpu_better_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fsspec
!pip install gcsfs

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.regularizers import l2

**Activate TPU accelerator**

In [ ]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy

In [ ]:
strategy = auto_select_accelerator()

INFO:tensorflow:Initializing the TPU system: grpc://10.123.171.50:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.123.171.50:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU: grpc://10.123.171.50:8470
Running on 8 replicas


**Image Decoder Function**

In [ ]:
def build_decoder(with_labels=True, target_size=(331,331), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode

**Data Augmentation Function**

In [ ]:
def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_contrast(img, 0.2, 0.4)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment

**Creating Dataset Function**

In [ ]:
def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

**Google Cloud Bucket Link**

In [ ]:
GCS_DS_PATH = 'gs://kds-e118bcdb309cf88b7f9e4a96ee84997123a5781b886180ffc13d3fc9'

**Read the CSV file**


In [ ]:
df = pd.read_csv(GCS_DS_PATH +'/train.csv')
sub_df = pd.read_csv(GCS_DS_PATH +'/sample_submission.csv')

In [ ]:
df

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3
...,...,...
21392,999068805.jpg,3
21393,999329392.jpg,3
21394,999474432.jpg,1
21395,999616605.jpg,4


In [ ]:
paths = GCS_DS_PATH + "/train_images/" + df['image_id']
test_paths = GCS_DS_PATH + "/test_images/" + sub_df['image_id']

In [ ]:
labels = df["label"]

**Train test split**

In [ ]:
(
    train_paths, valid_paths, 
    train_labels, valid_labels
) = train_test_split(paths, labels, test_size=0.2, random_state=42)

In [ ]:
img_size = 331
batch_size = strategy.num_replicas_in_sync * 16

In [ ]:
decoder = build_decoder(with_labels=True, target_size=(img_size, img_size))
test_decoder = build_decoder(with_labels=False, target_size=(img_size, img_size))

train_dataset = build_dataset(
    train_paths, train_labels, bsize=batch_size, decode_fn=decoder
)

valid_dataset = build_dataset(
    valid_paths, valid_labels, bsize=batch_size, decode_fn=decoder,
    repeat=False, shuffle=False, augment=False
)

test_dataset = build_dataset(
    test_paths, cache=False, bsize=batch_size, decode_fn=test_decoder,
    repeat=False, shuffle=False, augment=False
)

In [ ]:
train_dataset

<PrefetchDataset shapes: ((None, 331, 331, 3), (None,)), types: (tf.float32, tf.int64)>

In [ ]:
with strategy.scope():
    model = tf.keras.Sequential([
                                   tf.keras.applications.InceptionResNetV2(
          include_top=False,
      weights="imagenet",
      input_shape=(331,331,3)),
      tf.keras.layers.GlobalAveragePooling2D(),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dense(64, kernel_regularizer='l2'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dense(32, kernel_regularizer='l2'),
      tf.keras.layers.Dropout(0.3),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dense(5, activation='softmax')
      ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
    model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 9, 9, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 1536)              0         
_________________________________________________________________
batch_normalization_1015 (Ba (None, 1536)              6144      
_________________________________________________________________
dense_9 (Dense)              (None, 64)                98368     
_________________________________________________________________
batch_normalization_1016 (Ba (None, 64)                256       
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)               

In [32]:
steps_per_epoch = train_paths.shape[0] // batch_size
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'Cassava_model_new.h5', save_best_only=True, monitor='val_loss', mode='min')
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", patience=2, min_lr=1e-3, mode='min')

In [ ]:
history = model.fit(
    train_dataset, 
    epochs=20,
    steps_per_epoch=steps_per_epoch,callbacks = [checkpoint, lr_reducer],
    validation_data=valid_dataset)

Epoch 1/20
133/133 [==============================] - 300s 656ms/step - loss: 2.4425 - acc: 0.6067 - val_loss: 1.4298 - val_acc: 0.6488
Epoch 2/20
133/133 [==============================] - 57s 432ms/step - loss: 1.0271 - acc: 0.7590 - val_loss: 1.0766 - val_acc: 0.6815
Epoch 3/20
133/133 [==============================] - 58s 434ms/step - loss: 0.7594 - acc: 0.7986 - val_loss: 241.0663 - val_acc: 0.1091
Epoch 4/20
133/133 [==============================] - 59s 447ms/step - loss: 0.6259 - acc: 0.8219 - val_loss: 0.9829 - val_acc: 0.7332
Epoch 5/20
133/133 [==============================] - 58s 435ms/step - loss: 0.5468 - acc: 0.8380 - val_loss: 0.6728 - val_acc: 0.7886
Epoch 6/20
133/133 [==============================] - 58s 435ms/step - loss: 0.5051 - acc: 0.8510 - val_loss: 1.9515 - val_acc: 0.4963
Epoch 7/20
133/133 [==============================] - 58s 436ms/step - loss: 0.4804 - acc: 0.8508 - val_loss: 1.1366 - val_acc: 0.5897
Epoch 8/20
133/133 [==============================] 